In [ ]:
# Kaggle instalaltion in Colab # https://www.kaggle.com/general/74235
!pip install kaggle
!pip install transformers
!pip install colorama

     |████████████████████████████████| 2.5MB 14.3MB/s 
     |████████████████████████████████| 3.3MB 51.3MB/s 
     |████████████████████████████████| 901kB 44.2MB/s 


In [ ]:
# Here I'll use the kaggle dataset from Commolit Readability Competition https://www.kaggle.com/c/commonlitreadabilityprize/data
# At first let me Mount my drive to get the dataset
HOME_PATH = "/content"
%cd "$HOME_PATH"
from google.colab import drive 
drive.mount("/content/drive")
# Kaggle dataset path
KAGGLE_PATH = "/content/drive/My Drive/Data Science/data/"
DATASET_PATH = "/content/drive/My Drive/Data Science/data/commonlitreadabilityprize/"

/content
Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
!cp "/content/drive/My Drive/Data Science/data/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c commonlitreadabilityprize

  0% 0.00/108 [00:00<?, ?B/s]
100% 108/108 [00:00<00:00, 246kB/s]
  0% 0.00/6.79k [00:00<?, ?B/s]
100% 6.79k/6.79k [00:00<00:00, 6.81MB/s]
  0% 0.00/1.13M [00:00<?, ?B/s]
100% 1.13M/1.13M [00:00<00:00, 37.1MB/s]


In [ ]:
!mkdir input
!unzip train.csv.zip -d input
#!unzip file.zip -d input

Archive:  train.csv.zip
  inflating: input/train.csv         


In [ ]:
import os
import gc
import sys
import cv2
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold,StratifiedKFold

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,Sampler

from transformers import (AutoModel, AutoTokenizer,
                          AutoConfig ,AutoModelForSequenceClassification)

from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

In [ ]:
train_data = pd.read_csv('input/train.csv')
test_data = pd.read_csv('input/test.csv')
#sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
config = {
    'learning_rate':2e-5,
    'batch_size':16,
    'epochs':3,
    'nfolds':5,
    'seed':42,
    'max_len':256,
}

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])

In [ ]:
class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer):
        self.excerpt = df['excerpt'].to_numpy()
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],return_tensors='pt',
                                max_length=config['max_len'],
                                padding='max_length',truncation=True)
        return encode
    
    def __len__(self):
        return len(self.excerpt)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))

        score = self.V(att)

        attention_weights = torch.softmax(score, dim=1)

        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

In [ ]:
class Model(nn.Module):
    def __init__(self,model_path):
        super(Model,self).__init__()
        self.roberta = AutoModel.from_pretrained(model_path) 
        self.config = AutoConfig.from_pretrained(model_path)
        self.head = AttentionHead(self.config.hidden_size,self.config.hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(self.config.hidden_size,1)

    def forward(self,**xb):
        x = self.roberta(**xb)[0]
        x = self.head(x)
        x = self.dropout(x)
        x = self.linear(x)
        return x

In [ ]:
def get_prediction(df,path,model_path,device='cuda'):        
    model = Model(model_path)
    model.load_state_dict(torch.load(path,map_location=device))
    model.to(device)
    model.eval()
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    test_ds = CLRPDataset(df,tokenizer)
    test_dl = DataLoader(test_ds,
                        batch_size = config["batch_size"],
                        shuffle=False,
                        num_workers = 4,
                        pin_memory=True)
    
    predictions = list()
    for i, (inputs) in tqdm(enumerate(test_dl)):
        inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
        outputs = model(**inputs)
        outputs = outputs.cpu().detach().numpy().ravel().tolist()
        predictions.extend(outputs)
        
    torch.cuda.empty_cache()
    return np.array(predictions)

In [ ]:
pred1 = get_prediction(test_data,f'{DATASET_PATH}/clr-roberta/model0/model0.bin',f'{DATASET_PATH}/clrp_roberta_base')
pred2 = get_prediction(test_data,'../input/clr-roberta/model1/model1.bin','../input/clrp-roberta-base/clrp_roberta_base')
pred3 = get_prediction(test_data,'../input/clr-roberta/model2/model2.bin','../input/clrp-roberta-base/clrp_roberta_base')
pred4 = get_prediction(test_data,'../input/clr-roberta/model3/model3.bin','../input/clrp-roberta-base/clrp_roberta_base')
pred5 = get_prediction(test_data,'../input/clr-roberta/model4/model4.bin','../input/clrp-roberta-base/clrp_roberta_base')

predictions1 = (pred1 + pred2 + pred3 + pred4 + pred5)/5

In [ ]:
pred1 = get_prediction(test_data,'../input/clrp-roberta-base-models-tpu/model0/model0.bin','../input/clrp-roberta-base/clrp_roberta_base')
pred2 = get_prediction(test_data,'../input/clrp-roberta-base-models-tpu/model1/model1.bin','../input/clrp-roberta-base/clrp_roberta_base')
pred3 = get_prediction(test_data,'../input/clrp-roberta-base-models-tpu/model2/model2.bin','../input/clrp-roberta-base/clrp_roberta_base')
pred4 = get_prediction(test_data,'../input/clrp-roberta-base-models-tpu/model3/model3.bin','../input/clrp-roberta-base/clrp_roberta_base')
pred5 = get_prediction(test_data,'../input/clrp-roberta-base-models-tpu/model4/model4.bin','../input/clrp-roberta-base/clrp_roberta_base')
predictions2 = (pred1 + pred2 + pred3 + pred4 + pred5)/5

In [ ]:
sample['target'] = (predictions1 + predictions2)/2
sample.to_csv('submission.csv',index=False)

In [ ]:
sample